# Predicting Customer Churn for Telco Systems <a class="tocSkip">

## Notebook 3/4 - Pre-processing <a class="tocSkip">

by: __Neel Chudasama__

__Brainstation Data Science__

This notebook contains the steps I took to ensure my data is as cleaned as possible, so that the data that fit my models is as accurate as possible. I will be binarising columns, conducting statistical tests and removing/creating features.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Split" data-toc-modified-id="Data-Split-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Split</a></span><ul class="toc-item"><li><span><a href="#Statistical-Tests" data-toc-modified-id="Statistical-Tests-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Statistical Tests</a></span></li></ul></li><li><span><a href="#Binarise-data" data-toc-modified-id="Binarise-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Binarise data</a></span></li><li><span><a href="#Correlation-Plot" data-toc-modified-id="Correlation-Plot-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Correlation Plot</a></span></li><li><span><a href="#Clustering" data-toc-modified-id="Clustering-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Clustering</a></span><ul class="toc-item"><li><span><a href="#Hierarchical-Clustering" data-toc-modified-id="Hierarchical-Clustering-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Hierarchical Clustering</a></span></li></ul></li><li><span><a href="#Feature-Creation" data-toc-modified-id="Feature-Creation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Feature Creation</a></span></li><li><span><a href="#Saving-Work" data-toc-modified-id="Saving-Work-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Saving Work</a></span></li><li><span><a href="#Conclusions" data-toc-modified-id="Conclusions-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Conclusions</a></span></li></ul></div>

In [ ]:
# Main Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from scipy import stats
plt.rcParams["figure.dpi"] = 300 # to ensure that plots are not blurry

import capstone_utils
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import linkage, dendrogram

In [ ]:
from importlib import reload
reload(capstone_utils)

In [ ]:
#Loading the data from the pkl file 
telco_df = joblib.load('Data/telco_df.pkl')

## Data Split

Firstly, as seen previously there are categorical and numerical columns in the dataframe. I want to split those up into their own lists:

In [ ]:
cat_list = list(telco_df.select_dtypes("object").columns)
num_list = list(telco_df.select_dtypes("number").columns)

### Statistical Tests

I want to conduct a Chi squared test on the categorical columns with the churn column. Given that the churn column is the independent variable which we are testing against so I removed that from the column. Furthermore, I removed the CustomerID column as this will have no relevance to the dataframe as it is just an identifier column.

A Chi-Squared test tells me whether data is distributed across multiple variables as expected, or whether there is a statistically significant relationship between two categorical variables.

For a Chi-Squared test for independence, our hypotheses are:

$H_0$: There is no relationship between the column and Churn. (They are independent.)

$H_1$: There is _some_ relationship between the column and Churn. (They are not independent.)

In [ ]:
chi_sq_list = cat_list[1:-1]
capstone_utils.chi_squared_t(chi_sq_list,telco_df)

Looking at the chi squared dataframe for the categorical columns, the gender and PhoneService column have p values higher than the threshold of 0.05%. For these two columns we fail to reject the null hypothesis that they are independent and should be removed from the dataframe as these two columns have no effect on the dependent variable. 

In [ ]:
telco_df.drop(['gender', 'PhoneService'], axis=1, inplace = True)

Furthermore, given that each specific row is a seperate customer there is no need to differentiate between each one with the customer ID, therefore I have chosen to remove that column from the dataframe as well. 

In [ ]:
telco_df.drop(['customerID'], axis=1, inplace = True)

Again, looking at the list of categorical columns. Many of these columns are string data types with entries for 'yes' or 'no'. These can be converted into numeric columns but still hold the categorical data. In certain columns they hold data for three options for example, MultipleLines has 'Yes', 'No' and 'No phone service'.

Given that I dropped columns and removed redundant ones, I need to update the categorical list. 

In [ ]:
cat_list = list(telco_df.select_dtypes("object").columns)

In [ ]:
#Sanity Check
telco_df

## Binarise data

In [ ]:
capstone_utils.binarise_data(cat_list,telco_df)

With the help of the binarise_data function, I was able to convert the yes or no columns into binarised which were 0 or 1. 

Converting the columns to binary is just as easy for columns with three entries as well. Nearly all the columns with three entries have yes, no or no internet service. In the case of no internet service, this can be counted as no given that the customer needs internet service to be able to use the service. 

However, in the case of Internet service and contract it is a bit different and harder to convert. The three entries are different types of internet services, DSL, Fiber optic or none for the internet service column and different type of contract payments for contract. I shall leave, the Internet Service and contract column as it is for the time being. Furthermore, there is a slight variation for the MultipleLines column, instead of having "No internet service" it has "No phone service" so this change was accounted for.

In [ ]:
#Finaly sanity check 
telco_df.head(5)

Finally, I need to convert InternetService, Contract and PaymentMethod columns to dummy variables. These three columns contain categorical data that I cannot convert to numerical data so these columns need to be split up into their own designated column.

In [ ]:
telco_df[["InternetService","Contract","PaymentMethod"]].head(20)

In [ ]:
dummies = ["InternetService","Contract","PaymentMethod"]

telco_df = capstone_utils.dummy_variables(dummies, telco_df)

In [ ]:
#Sanity Check
telco_df.head(5)

## Correlation Plot

In [ ]:
capstone_utils.heat_map(telco_df)

There are a few columns which are very highly correlated, I want to remove these columns from the dataframe as this will help my models perform better. Colinearity between independent variables is bad for modelling because it undermines the statistical significance of a specific independent variable. Therfore, they need to be removed. 

The two columns which have multiple high correlations in them are tenure and MonthlyCharges. So these are the two that will be removed. If I chose to remove TotalCharges although that would remove the highest correlation but it would still leave many other correlations. I tried to remove the columns that contained the highest number of correlations. 

In [ ]:
telco_df = telco_df.drop(columns = ['tenure','MonthlyCharges'])

In [ ]:
#Sanity Check
telco_df.head(5)

## Clustering

Finally, for the last stage of my EDA I want to use clustering to determine if there are any groups within my data. For example, are customers who had dependents signing up for Streaming TV services more or whether there customers who signed up for Tech Support and Online Backup churned.

I will scale the data given that distances are highly important for clustering and because different features are in different sizing. Given that most of my data is in binary format, I have chosen to use a minmax scaler. 

From my EDA, I was able to determine that the data which were not categorical columns did not follow a normal distribution. Therefore, I determined that a MinMaxScaler would perform better than the standard scaler when your features have non-normal distributions. a minmax scaler is not distorting the actual distribution of the features, instead it is just reducing the range to be between 0 and 1. 

In [ ]:
X = telco_df

In [ ]:
X_mm = capstone_utils.minmax_clus_data(X)
telco_df_scaled = pd.DataFrame(data=X_mm, columns=X.columns)

In [ ]:
#Sanity Check
telco_df_scaled.head(5)

I want to determine if I can I am to determine the amount of clusters myself, from the data and EDA, I feel there will be a large amount of clusters. Therefore, I have chosen 10 clusters, with the help of PCA and t-SNE for the visualisation process - I can determine if this was the correct amount of clusters or not.

In [ ]:
capstone_utils.plot_pca(telco_df_scaled, 10)

There does seem to be some structure in a few of the plots, it could be determine that there are three or four clusters. However, the plots don't provide enough evidence for me to confirm that is the case. Given that PCA doesn't take distance into account when creating the plots, using t-SNE may be a better method of visualising the clusters. 

The function below uses tSNE, the default number of components I have chosen is 3 because anything larger than that will trigger an error. This poses another set of problems if there are more than 3, however, the results from the tSNE may produce better results than the PCA plot. 

In [ ]:
capstone_utils.plot_tsne(telco_df_scaled,2)

Looking at the results, I could determine that there are around two big clusters however it is hard to confirm whether that is the case. Therefore, I will explore K-Means, DBSCAN and Agglomerative Clustering to see if I can get more concrete results.

Firstly, with K-Means I have chosen a range of 2-50 clusters as my input. I chose such a large range as I felt that this covered the whole range and would account for both ends of the spectrum. Furthermore, during previous iterations I saw that the silhouette score rose when I had the upper limit at 20 so I wanted to account for this rise.

In [ ]:
capstone_utils.kmeans_range_clustering(2,50,telco_df_scaled)

A silhoutte score means greater seperation from each of the clusters, from the data it can be determined that 6 clusters is the best number. However, the silhouette score is quite low implying that the clusters are not seperated by much - this is noted and I will see if I can increase the cluster distance.

I will now input 6 as the number of clusters in my final model and plot the results below.

In [ ]:
capstone_utils.final_kmeans(6,telco_df_scaled)

I want to see if my hypothesis is right about there being 6 clusters in my data, to do so I will conduct Heirarchical clustering.

### Hierarchical Clustering

In [ ]:
capstone_utils.dendogram_clus(telco_df_scaled)

Looking at the plot above this shows that there are two groups that could be easily seperate from each other but this is at the top level. I could argue that there could be 6 clusters from evidence provided earlier but with the help of Agglomerative clustering I should be able to get a finite answer.

In [ ]:
capstone_utils.agglo_range_clust(2,11,telco_df_scaled)

In [ ]:
capstone_utils.final_agglo_custering(6,telco_df_scaled)

Looking at the plots above, this confirms the my earlier results that there 6 clusters, which have been viusalised with different colours. I will now input this into my final kmeans model and determine what the behaviour of each type of group is. 

In [ ]:
telco_df_final = capstone_utils.plot_kmean_relative_means(6,telco_df_scaled)

Looking at the Churn column especially, people in group 4 churned a large amount. I can grasp their pattern of behaviour from the other columns. For example, those in 4 also had a month-to-month contract, payed by electronic check, had paperlessbilling and payed for fiber optic. 

Looking at people in the purple group who churned the least, they had a partner, payed for online security, had two year contracts, payed for device protection and had DSL internet service. 

I am able to categorise that people who fall in these two groups and if they pay for these type of services are more likely to churn or not churn at a higher rate. This is incredibly valuable information in the future if there are customers who join the service and fall in specific group they can be categorised and their propensity to churn can be determined.

## Feature Creation

We now have an extra column which is called the kmeans_label column, this groups each customer into a specific group which was pre-determined by the clustering model. I will create extra features from that column with the use of dummy variables and this will aid my models in the classification process. But first, I need to remove the column from the telco_df_final dataframe and add it to the telco_df dataframe as this contains the target variable column - Churn.

In [ ]:
#Sanity Check
telco_df_final

In [ ]:
telco_df['kmeans_labels']=telco_df_final['kmeans_labels']

In [ ]:
telco_df = pd.get_dummies(telco_df, columns=['kmeans_labels'])

In [ ]:
#Sanity Check
telco_df

## Saving Work

Save the cleaned datagrame in a pkl file in my data folder

In [ ]:
joblib.dump(telco_df, 'data/final_telco_df.pkl')

## Conclusions

This notebook contains all the necessary steps I took to ensure that my dataframe was clean so that I could apply the modelling steps with ease and get the most accurate models possible. 

I have been able to remove features from the dataframe to ensure that there was no colinearity between variables. Furthermore, I was able to create more features from the dataframe to ensure that my models have as much statistically significant data as possible. 

The following notebook contains the steps I took for apply models to my dataframe.